# Bonus Lab — RMS: The Full Story
## What RMS really means, why it matters, and how to calculate it for any signal

---

**Prerequisites:** Module 00, Lesson 02 (AC vs DC and Signals) — the RMS section.

**What you'll learn:**
- The general RMS formula that works for *any* waveform
- RMS of signals with DC offsets — and why "total RMS" ≠ "AC RMS"
- RMS of PWM signals at different duty cycles
- RMS of rectified and clipped signals
- How RMS values combine when signals are added together
- Crest factor and form factor — the numbers behind meter accuracy
- Exactly how average-responding and True RMS meters work inside
- Where RMS shows up on datasheets (capacitor ripple, audio power, noise)

**Equipment:** Fnirsi 2C53T oscilloscope + signal generator, Klein MM300 multimeter

**Also useful:** AC coupling circuit from Bonus Lab 00 (for Experiment 1)

**Estimated time:** 30–60 minutes

In [ ]:
# Setup — run this cell first
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as sig
%matplotlib inline

plt.rcParams.update({
    'figure.figsize': (10, 5),
    'axes.grid': True,
    'font.size': 12,
    'lines.linewidth': 2,
    'grid.alpha': 0.3,
})
print("Setup complete.")

---
## 1. Quick Recap

In Module 00, we established that **RMS (Root Mean Square)** answers the question: *"What DC voltage would deliver the same power to a resistor?"* For a sine wave, $V_{rms} = V_{peak} / \sqrt{2}$. For other waveforms, the ratio is different.

That introduction covered the basics. This lab goes deeper — into the signals you'll actually encounter in real circuits, and the measurement traps waiting for you when those signals aren't textbook sine waves.

---
## 2. The General Formula

The $V_{peak}/\sqrt{2}$ shortcut only works for sine waves. The **general** RMS definition works for any waveform:

$$V_{rms} = \sqrt{\frac{1}{T}\int_0^T v(t)^2 \, dt}$$

Read it right-to-left: **square** every instantaneous voltage, take the **mean** over one period, then take the **root**.

In practice — whether you're writing code or a meter is doing it internally — you work with discrete samples:

$$V_{rms} = \sqrt{\frac{1}{N}\sum_{i=1}^{N} v_i^2}$$

This is just `np.sqrt(np.mean(v**2))`. It works for any signal — periodic, aperiodic, noisy, distorted, whatever. If you can sample it, you can compute its RMS.

---
## 3. RMS of Standard Waveforms — Showing the Math

Module 00 gave the results. Here are the derivations:

### Sine wave

For $v(t) = V_p \sin(\omega t)$:

$$V_{rms}^2 = \frac{1}{T}\int_0^T V_p^2 \sin^2(\omega t)\, dt = V_p^2 \cdot \frac{1}{2} = \frac{V_p^2}{2}$$

because the average of $\sin^2$ over a full cycle is exactly $\frac{1}{2}$. So $V_{rms} = V_p / \sqrt{2}$.

### Square wave

For a symmetric square wave, $v(t) = \pm V_p$. Then $v(t)^2 = V_p^2$ always. The mean of a constant is that constant. So $V_{rms} = V_p$. Trivial.

### Triangle wave

A triangle wave rises linearly from $-V_p$ to $V_p$ over half a period, then falls back. Over one quarter cycle (the rising portion from 0 to $V_p$), $v(t) = V_p \cdot (4t/T)$ for $0 \leq t \leq T/4$.

By symmetry, we can compute the mean of $v^2$ over just this quarter cycle:

$$V_{rms}^2 = \frac{4}{T}\int_0^{T/4} V_p^2 \left(\frac{4t}{T}\right)^2 dt = V_p^2 \cdot \frac{1}{3}$$

So $V_{rms} = V_p / \sqrt{3}$. The sawtooth wave gives the same result — it has the same time-averaged squared value.

In [ ]:
# --- Visualize v(t), v²(t), mean of v², and RMS for each standard waveform ---

f = 1000
T = 1 / f
t = np.linspace(0, 2 * T, 4000)  # 2 cycles
V_p = 3.0

waveforms = [
    ('Sine',     V_p * np.sin(2 * np.pi * f * t),                        V_p / np.sqrt(2)),
    ('Square',   V_p * sig.square(2 * np.pi * f * t),                    V_p),
    ('Triangle', V_p * sig.sawtooth(2 * np.pi * f * t, width=0.5),      V_p / np.sqrt(3)),
    ('Sawtooth', V_p * sig.sawtooth(2 * np.pi * f * t),                 V_p / np.sqrt(3)),
]

fig, axes = plt.subplots(len(waveforms), 2, figsize=(14, 12))

for i, (name, v, v_rms_theory) in enumerate(waveforms):
    v_sq = v ** 2
    mean_v_sq = np.mean(v_sq)
    v_rms_num = np.sqrt(mean_v_sq)
    
    # Left: v(t) with RMS lines
    axes[i, 0].plot(t * 1000, v, color='#2980b9')
    axes[i, 0].axhline(y=v_rms_num, color='orange', linestyle='--', label=f'V_rms = {v_rms_num:.3f}V')
    axes[i, 0].axhline(y=-v_rms_num, color='orange', linestyle='--')
    axes[i, 0].axhline(y=0, color='black', linewidth=0.5)
    axes[i, 0].set_ylabel('V')
    axes[i, 0].set_title(f'{name}: v(t)')
    axes[i, 0].set_ylim(-V_p * 1.3, V_p * 1.3)
    axes[i, 0].legend(fontsize=9, loc='upper right')
    
    # Right: v²(t) with mean line
    axes[i, 1].plot(t * 1000, v_sq, color='#e74c3c', linewidth=1.5)
    axes[i, 1].axhline(y=mean_v_sq, color='orange', linestyle='--',
                        label=f'Mean(v²) = {mean_v_sq:.2f}V²')
    axes[i, 1].set_ylabel('V²')
    axes[i, 1].set_title(f'{name}: v²(t) — RMS = √{mean_v_sq:.2f} = {v_rms_num:.3f}V')
    axes[i, 1].set_ylim(0, V_p**2 * 1.2)
    axes[i, 1].legend(fontsize=9, loc='upper right')

axes[-1, 0].set_xlabel('Time (ms)')
axes[-1, 1].set_xlabel('Time (ms)')
plt.tight_layout()
plt.show()

# Summary table
print(f"{'Waveform':<12} {'V_rms (numeric)':>15} {'V_rms (theory)':>15} {'V_rms/V_peak':>13} {'Theory ratio':>13}")
print("-" * 70)
for name, v, v_rms_theory in waveforms:
    v_rms_num = np.sqrt(np.mean(v**2))
    print(f"{name:<12} {v_rms_num:>13.4f} V {v_rms_theory:>13.4f} V {v_rms_num/V_p:>13.4f} {v_rms_theory/V_p:>13.4f}")

---
## 4. RMS of Signals with DC Offset

This is where it gets practical. The Fnirsi outputs 0V-to-3V signals — they have a DC component. What is the RMS of the *total* signal vs the RMS of just the AC part?

Any signal can be decomposed into a DC component and an AC component:

$$v(t) = V_{DC} + v_{AC}(t)$$

where $V_{DC}$ is the average (mean) of the signal, and $v_{AC}(t)$ is the deviation from that average.

A key identity:

$$V_{rms,total}^2 = V_{DC}^2 + V_{rms,AC}^2$$

This works because when you expand $(V_{DC} + v_{AC})^2$, the cross-term $2 \cdot V_{DC} \cdot v_{AC}$ averages to zero (since $v_{AC}$ has zero mean by definition).

### Why this matters

- **Total RMS** tells you the total power delivered. A 0–3V sine wave across a 100Ω resistor delivers $V_{rms,total}^2 / R$, not $V_{rms,AC}^2 / R$.
- **AC RMS** is what AC-coupled instruments report — the oscilloscope in AC mode, the multimeter in AC mode (if it properly blocks DC).
- They're **different numbers** for the same signal. Confusing the two is a common source of measurement errors.

In [ ]:
# --- DC + AC decomposition of the Fnirsi's unipolar signal ---

f = 1000
t = np.linspace(0, 3/f, 3000)
amplitude = 3.0  # Fnirsi amplitude (0V to 3V)

# Total signal: 0 to 3V
v_total = (amplitude / 2) + (amplitude / 2) * np.sin(2 * np.pi * f * t)

# Decompose
V_dc = np.mean(v_total)
v_ac = v_total - V_dc

# RMS calculations
V_rms_total = np.sqrt(np.mean(v_total**2))
V_rms_dc = V_dc  # RMS of a constant = the constant
V_rms_ac = np.sqrt(np.mean(v_ac**2))

fig, axes = plt.subplots(3, 1, figsize=(12, 9), sharex=True)

axes[0].plot(t * 1000, v_total, color='#2980b9')
axes[0].axhline(y=V_rms_total, color='orange', linestyle='--',
                label=f'V_rms(total) = {V_rms_total:.3f}V')
axes[0].set_ylabel('V')
axes[0].set_title('Total signal: 0V to 3V (unipolar)')
axes[0].set_ylim(-2, 4)
axes[0].legend()

axes[1].axhline(y=V_dc, color='#e74c3c', linewidth=2,
                label=f'V_DC = {V_dc:.3f}V (this IS the RMS of the DC component)')
axes[1].set_ylabel('V')
axes[1].set_title('DC component')
axes[1].set_ylim(-2, 4)
axes[1].legend()

axes[2].plot(t * 1000, v_ac, color='#27ae60')
axes[2].axhline(y=V_rms_ac, color='orange', linestyle='--',
                label=f'V_rms(AC) = {V_rms_ac:.3f}V')
axes[2].axhline(y=-V_rms_ac, color='orange', linestyle='--')
axes[2].axhline(y=0, color='black', linewidth=0.5)
axes[2].set_ylabel('V')
axes[2].set_xlabel('Time (ms)')
axes[2].set_title('AC component (DC removed)')
axes[2].set_ylim(-2, 4)
axes[2].legend()

plt.tight_layout()
plt.show()

print(f"DC component:    V_dc      = {V_dc:.4f} V")
print(f"AC component:    V_rms(AC) = {V_rms_ac:.4f} V")
print(f"Total signal:    V_rms(total) = {V_rms_total:.4f} V")
print()
print(f"Verify: V_dc² + V_rms(AC)² = {V_dc**2:.4f} + {V_rms_ac**2:.4f} = {V_dc**2 + V_rms_ac**2:.4f}")
print(f"        V_rms(total)²       = {V_rms_total**2:.4f}")
print(f"        Match: {np.isclose(V_dc**2 + V_rms_ac**2, V_rms_total**2)}")
print()
print(f"A multimeter in AC mode (if properly AC-coupled) reads: {V_rms_ac:.3f}V")
print(f"The actual total RMS is: {V_rms_total:.3f}V")
print(f"These are NOT the same number!")

---
## 5. RMS of PWM Signals

**PWM (Pulse Width Modulation)** is everywhere — LED dimmers, motor controllers, power converters, audio class-D amplifiers. The signal is either fully ON ($V_{high}$) or fully OFF (0V), and the **duty cycle** $D$ controls what fraction of the time it's ON.

For a PWM signal that's $V_{high}$ for fraction $D$ of the period and 0V for the rest:

$$V_{rms} = V_{high} \cdot \sqrt{D}$$

The derivation is straightforward: $v^2 = V_{high}^2$ for fraction $D$ of the time, and $v^2 = 0$ for the rest. The mean of $v^2$ is $D \cdot V_{high}^2$. Take the root: $V_{rms} = V_{high} \sqrt{D}$.

| Duty cycle | V_rms / V_high | Meaning |
|-----------|----------------|----------|
| 10% | 0.316 | LED very dim |
| 25% | 0.500 | Quarter power |
| 50% | 0.707 | Half power (same ratio as sine!) |
| 75% | 0.866 | Three-quarter power |
| 100% | 1.000 | Full DC — it's just ON |

Note the common mistake: the **average** voltage of a 50% PWM signal is $V_{high}/2$, but the **RMS** is $V_{high}/\sqrt{2}$. Average and RMS are only the same for DC. For power calculations, you need RMS.

In [ ]:
# --- PWM: RMS vs duty cycle ---

V_high = 5.0
f = 1000
T = 1 / f
t = np.linspace(0, 3 * T, 6000)

duty_cycles = [0.10, 0.25, 0.50, 0.75, 0.90]

fig, axes = plt.subplots(len(duty_cycles), 1, figsize=(12, 12), sharex=True)

for ax, D in zip(axes, duty_cycles):
    v = V_high * (sig.square(2 * np.pi * f * t, duty=D) + 1) / 2  # 0 to V_high
    v_rms = np.sqrt(np.mean(v**2))
    v_avg = np.mean(v)
    v_rms_theory = V_high * np.sqrt(D)
    
    ax.plot(t * 1000, v, color='#2980b9', linewidth=1.5)
    ax.axhline(y=v_rms, color='orange', linestyle='--', linewidth=1.5,
               label=f'V_rms = {v_rms:.2f}V')
    ax.axhline(y=v_avg, color='red', linestyle=':', linewidth=1.5,
               label=f'V_avg = {v_avg:.2f}V')
    ax.set_ylabel('V')
    ax.set_title(f'D = {D*100:.0f}% — V_rms = {V_high}V × √{D} = {v_rms_theory:.2f}V', fontsize=11)
    ax.set_ylim(-0.5, V_high * 1.3)
    ax.legend(fontsize=9, loc='right')

axes[-1].set_xlabel('Time (ms)')
plt.tight_layout()
plt.show()

# RMS vs duty cycle curve
D_range = np.linspace(0, 1, 200)
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(D_range * 100, V_high * np.sqrt(D_range), color='#2980b9', linewidth=2,
        label='V_rms = V_high × √D')
ax.plot(D_range * 100, V_high * D_range, color='red', linestyle=':', linewidth=2,
        label='V_avg = V_high × D (NOT the same!)')
ax.set_xlabel('Duty Cycle (%)')
ax.set_ylabel('Voltage (V)')
ax.set_title(f'PWM: RMS vs Average — V_high = {V_high}V')
ax.legend(fontsize=11)
plt.tight_layout()
plt.show()

print("The gap between the curves shows the error from using average instead of RMS.")
print("At 50% duty: V_avg = 2.5V but V_rms = 3.54V — a 41% difference.")

---
## 6. RMS of Rectified Signals

Rectifiers convert AC to (pulsating) DC — they're inside every AC-to-DC power supply. What's the RMS of a rectified sine wave?

### Half-wave rectified

Only the positive half-cycles pass through; the negative half-cycles are replaced with 0V. The signal is $V_p \sin(\omega t)$ for half the cycle and 0V for the other half. So $v^2$ is nonzero for only half the cycle:

$$V_{rms} = \frac{V_p}{2}$$

### Full-wave rectified

Both half-cycles are made positive — the negative half is flipped. Now $v(t) = |V_p \sin(\omega t)|$. But here's the insight: $v^2(t) = V_p^2 \sin^2(\omega t)$ regardless of the sign. Squaring already makes everything positive. So:

$$V_{rms} = \frac{V_p}{\sqrt{2}}$$

**The same as the unrectified sine wave!** Full-wave rectification doesn't change the RMS because squaring already ignores the sign. This is a beautiful demonstration of why RMS is the right way to measure AC power — it's insensitive to which direction the current flows.

In [ ]:
# --- Rectified sine waves: v(t) and v²(t) ---

f = 60
t = np.linspace(0, 2/f, 4000)
V_p = 170  # US mains peak

v_sine = V_p * np.sin(2 * np.pi * f * t)
v_half = np.maximum(v_sine, 0)  # half-wave rectified
v_full = np.abs(v_sine)         # full-wave rectified

signals = [
    ('Original sine', v_sine, '#2980b9'),
    ('Half-wave rectified', v_half, '#e74c3c'),
    ('Full-wave rectified', v_full, '#27ae60'),
]

fig, axes = plt.subplots(3, 2, figsize=(14, 10))

for i, (name, v, color) in enumerate(signals):
    v_rms = np.sqrt(np.mean(v**2))
    v_sq = v**2
    
    axes[i, 0].plot(t * 1000, v, color=color)
    axes[i, 0].axhline(y=0, color='black', linewidth=0.5)
    axes[i, 0].set_ylabel('V')
    axes[i, 0].set_title(f'{name} — V_rms = {v_rms:.1f}V')
    axes[i, 0].set_ylim(-V_p * 1.2, V_p * 1.2)
    
    axes[i, 1].plot(t * 1000, v_sq / 1000, color=color, linewidth=1.5)
    axes[i, 1].axhline(y=np.mean(v_sq) / 1000, color='orange', linestyle='--',
                        label=f'Mean(v²) = {np.mean(v_sq):.0f} V²')
    axes[i, 1].set_ylabel('kV²')
    axes[i, 1].set_title(f'{name} — v²(t)')
    axes[i, 1].set_ylim(0, V_p**2 / 1000 * 1.2)
    axes[i, 1].legend(fontsize=9)

axes[-1, 0].set_xlabel('Time (ms)')
axes[-1, 1].set_xlabel('Time (ms)')
plt.tight_layout()
plt.show()

print(f"Original sine:        V_rms = {np.sqrt(np.mean(v_sine**2)):.1f}V  (V_p/√2 = {V_p/np.sqrt(2):.1f}V)")
print(f"Half-wave rectified:  V_rms = {np.sqrt(np.mean(v_half**2)):.1f}V  (V_p/2 = {V_p/2:.1f}V)")
print(f"Full-wave rectified:  V_rms = {np.sqrt(np.mean(v_full**2)):.1f}V  (V_p/√2 = {V_p/np.sqrt(2):.1f}V)")
print()
print("Notice: full-wave v²(t) is IDENTICAL to original v²(t).")
print("Squaring already makes everything positive — rectification is redundant for RMS.")

---
## 7. RMS of Clipped Signals

When a sine wave clips — because an amplifier saturates, or a signal exceeds a supply rail — the peaks get flattened. The waveform starts looking more like a square wave.

Since a square wave has a higher RMS-to-peak ratio than a sine wave, **clipping increases RMS**. This is important for component ratings: if a signal clips unexpectedly, the power delivered to a load increases, potentially exceeding the component's rating.

In [ ]:
# --- Clipped sine wave: RMS increases with clipping ---

f = 1000
t = np.linspace(0, 2/f, 4000)
V_p = 5.0
v_sine = V_p * np.sin(2 * np.pi * f * t)

clip_levels = [5.0, 4.0, 3.0, 2.0, 1.0]  # clip at ±this voltage
colors = ['#2980b9', '#3498db', '#e67e22', '#e74c3c', '#c0392b']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for clip, color in zip(clip_levels, colors):
    v_clipped = np.clip(v_sine, -clip, clip)
    v_rms = np.sqrt(np.mean(v_clipped**2))
    ax1.plot(t * 1000, v_clipped, color=color, linewidth=1.5,
             label=f'Clip ±{clip}V → RMS={v_rms:.2f}V')

ax1.axhline(y=0, color='black', linewidth=0.5)
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Voltage (V)')
ax1.set_title('Sine wave with increasing clipping')
ax1.legend(fontsize=9)

# RMS vs clip level
clip_range = np.linspace(0.1, V_p, 200)
rms_values = []
for clip in clip_range:
    v_clipped = np.clip(v_sine, -clip, clip)
    rms_values.append(np.sqrt(np.mean(v_clipped**2)))

ax2.plot(clip_range, rms_values, color='#2980b9', linewidth=2)
ax2.axhline(y=V_p / np.sqrt(2), color='green', linestyle='--', alpha=0.5,
            label=f'Unclipped sine RMS ({V_p/np.sqrt(2):.2f}V)')
ax2.axhline(y=V_p, color='red', linestyle='--', alpha=0.5,
            label=f'Square wave RMS ({V_p:.1f}V)')
ax2.set_xlabel('Clip level (V)')
ax2.set_ylabel('V_rms')
ax2.set_title('RMS increases as clipping gets harder')
ax2.legend(fontsize=9)

plt.tight_layout()
plt.show()

print(f"Unclipped 5V sine: V_rms = {V_p/np.sqrt(2):.2f}V")
print(f"Clipped at ±3V:    V_rms = {np.sqrt(np.mean(np.clip(v_sine,-3,3)**2)):.2f}V (higher!)")
print(f"Clipped at ±1V:    V_rms = {np.sqrt(np.mean(np.clip(v_sine,-1,1)**2)):.2f}V (approaching square wave)")
print(f"Perfect square:    V_rms = {V_p:.2f}V")

---
## 8. Adding Signals — RMS Adds in Quadrature

If you have two signals at **different frequencies** (or two uncorrelated noise sources), their RMS values combine as:

$$V_{rms,total}^2 = V_{rms,1}^2 + V_{rms,2}^2$$

or equivalently: $V_{rms,total} = \sqrt{V_{rms,1}^2 + V_{rms,2}^2}$

This is called **quadrature addition** (like the Pythagorean theorem). It works because the cross-term $2 v_1(t) \cdot v_2(t)$, when averaged over time, goes to zero for signals at different frequencies — the positive and negative products cancel perfectly.

**Practical example:** You have a 3V_rms signal with 1V_rms of noise on top. The total RMS is:

$$V_{rms,total} = \sqrt{3^2 + 1^2} = \sqrt{10} \approx 3.16\text{V}$$

NOT $3 + 1 = 4\text{V}$. The noise contributes surprisingly little to the total power.

**Caveat:** This only works if the signals are at different frequencies (or uncorrelated). Two sine waves at the *same* frequency can add constructively or destructively depending on their phase — in that case the cross-term doesn't cancel.

In [ ]:
# --- Quadrature addition of RMS ---

f1, f2 = 1000, 5000  # Hz
t = np.linspace(0, 10/f1, 100000)  # many cycles for accuracy

V1_peak = 3.0 * np.sqrt(2)  # 3V RMS
V2_peak = 1.0 * np.sqrt(2)  # 1V RMS

v1 = V1_peak * np.sin(2 * np.pi * f1 * t)
v2 = V2_peak * np.sin(2 * np.pi * f2 * t)
v_total = v1 + v2

rms1 = np.sqrt(np.mean(v1**2))
rms2 = np.sqrt(np.mean(v2**2))
rms_total = np.sqrt(np.mean(v_total**2))
rms_quadrature = np.sqrt(rms1**2 + rms2**2)

# Show a few cycles
t_show = np.linspace(0, 3/f1, 3000)
v1_show = V1_peak * np.sin(2 * np.pi * f1 * t_show)
v2_show = V2_peak * np.sin(2 * np.pi * f2 * t_show)
v_total_show = v1_show + v2_show

fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

axes[0].plot(t_show * 1000, v1_show, color='#2980b9')
axes[0].set_ylabel('V')
axes[0].set_title(f'Signal 1: {f1} Hz, V_rms = {rms1:.2f}V')
axes[0].set_ylim(-7, 7)

axes[1].plot(t_show * 1000, v2_show, color='#e74c3c')
axes[1].set_ylabel('V')
axes[1].set_title(f'Signal 2: {f2} Hz, V_rms = {rms2:.2f}V')
axes[1].set_ylim(-7, 7)

axes[2].plot(t_show * 1000, v_total_show, color='#27ae60')
axes[2].axhline(y=rms_total, color='orange', linestyle='--',
                label=f'V_rms(total) = {rms_total:.3f}V')
axes[2].axhline(y=-rms_total, color='orange', linestyle='--')
axes[2].set_ylabel('V')
axes[2].set_xlabel('Time (ms)')
axes[2].set_title(f'Combined signal: V_rms = {rms_total:.3f}V')
axes[2].set_ylim(-7, 7)
axes[2].legend()

plt.tight_layout()
plt.show()

print(f"V_rms(signal 1) = {rms1:.3f}V")
print(f"V_rms(signal 2) = {rms2:.3f}V")
print(f"V_rms(combined) = {rms_total:.3f}V")
print()
print(f"Quadrature prediction: √({rms1:.3f}² + {rms2:.3f}²) = √({rms1**2:.3f} + {rms2**2:.3f}) = {rms_quadrature:.3f}V")
print(f"Linear addition:       {rms1:.3f} + {rms2:.3f} = {rms1 + rms2:.3f}V  ← WRONG")
print()
print(f"Quadrature matches: {np.isclose(rms_total, rms_quadrature, atol=0.01)}")

---
## 9. Crest Factor and Form Factor

Two dimensionless ratios that characterize waveform shape and determine how meters behave:

### Crest Factor

$$\text{Crest Factor} = \frac{V_{peak}}{V_{rms}}$$

This tells you how "peaky" a signal is relative to its RMS. A higher crest factor means the signal has brief high-amplitude peaks but lower sustained power.

- **Square wave:** CF = 1.0 — flat top, never exceeds RMS
- **Sine wave:** CF = √2 ≈ 1.414
- **Triangle wave:** CF = √3 ≈ 1.732
- **Audio music:** CF can be 10–20 — transient peaks far above the average level

Why it matters: component ratings. A capacitor or transistor might handle the RMS current fine, but the **peak** current during a transient could exceed its rating. Crest factor tells you how much headroom you need.

### Form Factor

$$\text{Form Factor} = \frac{V_{rms}}{V_{avg(rectified)}}$$

where $V_{avg(rectified)}$ is the average of $|v(t)|$ over one cycle.

This is the number that explains why average-responding meters are wrong for non-sine waveforms. The Klein MM300:
1. Internally measures $V_{avg(rectified)}$ (the average of $|v|$)
2. Multiplies by **1.111** (the form factor of a sine wave)
3. Displays the result as "V_rms"

This is correct *only if the signal is a sine wave*. For any other waveform, the form factor is different, and the displayed value is wrong.

| Waveform | Form Factor | MM300 error |
|----------|------------|------------|
| Sine | 1.111 | 0% (calibrated for this) |
| Square | 1.000 | Reads 11.1% high |
| Triangle | 1.155 | Reads 3.8% low |

In [ ]:
# --- Crest Factor, Form Factor, and meter error for all waveforms ---

f = 1000
t = np.linspace(0, 10/f, 100000)
V_p = 5.0

SINE_FORM_FACTOR = np.pi / (2 * np.sqrt(2))  # ≈ 1.1107

waveforms = {
    'Sine':     V_p * np.sin(2 * np.pi * f * t),
    'Square':   V_p * sig.square(2 * np.pi * f * t),
    'Triangle': V_p * sig.sawtooth(2 * np.pi * f * t, width=0.5),
    'Sawtooth': V_p * sig.sawtooth(2 * np.pi * f * t),
    'PWM 25%':  V_p * ((sig.square(2 * np.pi * f * t, duty=0.25) + 1) / 2),
}

print(f"V_peak = {V_p}V for all waveforms (except PWM which goes 0V to {V_p}V)")
print()
print(f"{'Waveform':<12} {'V_rms':>8} {'V_avg|v|':>9} {'Crest':>7} {'Form':>7} {'MM300 reads':>12} {'Error':>8}")
print("-" * 68)

for name, v in waveforms.items():
    v_rms = np.sqrt(np.mean(v**2))
    v_peak = np.max(np.abs(v))
    v_avg_rect = np.mean(np.abs(v))
    
    crest = v_peak / v_rms
    form = v_rms / v_avg_rect
    
    # What MM300 displays: V_avg_rect * sine_form_factor
    mm300_reading = v_avg_rect * SINE_FORM_FACTOR
    error_pct = (mm300_reading - v_rms) / v_rms * 100
    
    print(f"{name:<12} {v_rms:>6.3f}V {v_avg_rect:>7.3f}V {crest:>7.3f} {form:>7.3f} {mm300_reading:>10.3f}V {error_pct:>+7.1f}%")

print()
print("The MM300 assumes Form Factor = 1.111 (sine wave).")
print("For non-sine waveforms, the displayed value deviates from the true RMS.")

---
## 10. How Meters Actually Measure AC

Understanding the internals makes the errors above predictable instead of mysterious.

### Average-responding meter (Klein MM300)

1. The AC signal enters the meter.
2. A **precision rectifier** circuit converts it to $|v(t)|$ — all positive.
3. A **low-pass filter** (or integrating circuit) computes the average of $|v(t)|$. This is $V_{avg(rectified)}$.
4. The meter **multiplies by 1.111** (the form factor of a sine wave) and displays the result.

This is simple, cheap, and works well for its intended purpose: measuring AC mains and other sinusoidal signals. It falls apart for non-sinusoidal signals because the form factor is baked in.

### True RMS meter

**Analog approach (thermal converter):** The signal heats a tiny resistor. A thermocouple measures the temperature. Since heating is proportional to $V^2$, the temperature reading is proportional to the mean of $V^2$ — which is $V_{rms}^2$. The meter takes the square root and displays it. This works for any waveform because physics does the squaring.

**Digital approach (modern meters):** A fast ADC samples the signal. A DSP computes $\sqrt{\frac{1}{N}\sum v_i^2}$ — literally the discrete RMS formula. This is what `np.sqrt(np.mean(v**2))` does.

### AC-coupled vs AC+DC True RMS

Even True RMS meters come in two varieties:
- **AC-coupled True RMS:** Blocks DC with an internal capacitor, then measures RMS of the AC component only. Reports $V_{rms,AC}$.
- **AC+DC True RMS:** Measures RMS of the complete signal including DC. Reports $V_{rms,total}$.

Remember from Section 4: these are different numbers for signals with DC offset. The meter's manual will tell you which type it is.

In [ ]:
# --- Simulate what different meter types would read ---

f = 1000
t = np.linspace(0, 10/f, 100000)
V_p = 1.5  # AC peak (after coupling from 3V Fnirsi output)

SINE_FF = np.pi / (2 * np.sqrt(2))

test_signals = {
    'Sine':              V_p * np.sin(2 * np.pi * f * t),
    'Square':            V_p * sig.square(2 * np.pi * f * t),
    'Triangle':          V_p * sig.sawtooth(2 * np.pi * f * t, width=0.5),
    'PWM 25%\n(0 to 3V)': 3.0 * ((sig.square(2 * np.pi * f * t, duty=0.25) + 1) / 2),
    'Clipped sine\n(±1V clip)': np.clip(V_p * np.sin(2 * np.pi * f * t), -1, 1),
}

fig, ax = plt.subplots(figsize=(12, 6))

names = list(test_signals.keys())
x = np.arange(len(names))
width = 0.25

true_rms_vals = []
avg_resp_vals = []

for name, v in test_signals.items():
    true_rms = np.sqrt(np.mean(v**2))
    avg_resp = np.mean(np.abs(v)) * SINE_FF
    true_rms_vals.append(true_rms)
    avg_resp_vals.append(avg_resp)

bars1 = ax.bar(x - width/2, true_rms_vals, width, label='True RMS', color='#27ae60')
bars2 = ax.bar(x + width/2, avg_resp_vals, width, label='Average-responding (MM300)', color='#e74c3c')

# Add value labels
for bar in bars1:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            f'{bar.get_height():.3f}', ha='center', va='bottom', fontsize=9, color='#27ae60')
for bar in bars2:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            f'{bar.get_height():.3f}', ha='center', va='bottom', fontsize=9, color='#e74c3c')

ax.set_xticks(x)
ax.set_xticklabels(names, fontsize=10)
ax.set_ylabel('Reading (V)')
ax.set_title('What Different Meters Display for the Same Signals')
ax.legend(fontsize=11)

plt.tight_layout()
plt.show()

print("Green = correct (True RMS). Red = what the MM300 displays.")
print("They match only for sine waves. For everything else, the MM300 is off.")

---
## 11. Experiments

### Experiment 1: Measure RMS of Different Waveforms

**Goal:** Verify the meter error predictions from the previous section using your actual equipment.

**Setup:** Use the **AC coupling circuit from Bonus Lab 00** (1µF cap + 10kΩ resistor) so the MM300 receives a bipolar signal.

```
Fnirsi Signal Out ──┤├──┬──── MM300 (AC V mode)
                  1µF   │
                       10kΩ
                        │
                       GND ──── MM300 COM
```

**Steps:**

1. Generate each waveform at **1 kHz, 3V amplitude** on the Fnirsi.
2. For each, record the MM300 reading.
3. Also read V_pp from the oscilloscope (connected to the same node).

Fill in this table:

| Waveform | Scope V_pp | True V_rms (calculated) | MM300 reading | Expected MM300 | Error |
|----------|-----------|------------------------|---------------|----------------|-------|
| Sine | | | | 1.06V | ~0% |
| Square | | | | 1.67V | +11% |
| Triangle | | | | 1.44V | −4% |

The "True V_rms" column is computed from V_pp: for the AC-coupled output, V_peak = V_pp/2, then apply the appropriate ratio (sine: /√2, square: ×1, triangle: /√3).

---

### Experiment 2: PWM Duty Cycle vs RMS

**Goal:** Verify that $V_{rms} = V_{high} \times \sqrt{D}$ for PWM.

If the Fnirsi supports adjustable duty cycle on square waves:

1. Set square wave at 1 kHz, 3V amplitude.
2. Adjust duty cycle to 25%, 50%, 75%.
3. For each, read the oscilloscope V_pp and the MM300 AC reading.

Note: the MM300 readings will be wrong (average-responding meter + PWM = double trouble), but the oscilloscope measurements should confirm the waveform shape. You can compute the true RMS from the duty cycle and amplitude.

If the Fnirsi doesn't support duty cycle adjustment, this experiment becomes a calculation exercise — use the code cells above to predict values for different duty cycles.

In [ ]:
# --- Predicted readings for Experiment 1 ---

amplitude = 3.0  # Fnirsi amplitude setting
V_ac_peak = amplitude / 2  # After AC coupling: ±1.5V

SINE_FF = np.pi / (2 * np.sqrt(2))

# For each waveform, compute true RMS and predicted MM300 reading
print("Experiment 1: Predicted values (AC-coupled Fnirsi output)")
print("=" * 70)
print(f"Fnirsi amplitude: {amplitude}V → AC component: ±{V_ac_peak}V\n")

exp_waveforms = [
    ('Sine',     1 / np.sqrt(2), 2 / np.pi),
    ('Square',   1.0,            1.0),
    ('Triangle', 1 / np.sqrt(3), 1 / 2),
]

print(f"{'Waveform':<12} {'V_pp':>6} {'V_rms (true)':>13} {'V_avg|v|':>9} {'MM300 reads':>12} {'Error':>8}")
print("-" * 62)

for name, rms_ratio, avg_ratio in exp_waveforms:
    v_pp = 2 * V_ac_peak
    v_rms = V_ac_peak * rms_ratio
    v_avg_rect = V_ac_peak * avg_ratio
    mm300 = v_avg_rect * SINE_FF
    error = (mm300 - v_rms) / v_rms * 100
    print(f"{name:<12} {v_pp:>4.1f}V {v_rms:>11.3f}V {v_avg_rect:>7.3f}V {mm300:>10.3f}V {error:>+7.1f}%")

print()
print("Your MM300 readings should be close to the 'MM300 reads' column.")
print("The sine reading should be accurate. Square reads high. Triangle reads low.")

---
## 12. RMS on Datasheets

RMS shows up in component specifications far more often than you might expect. Knowing what it means in each context prevents costly mistakes.

### Capacitor ripple current

Electrolytic capacitors in power supplies experience AC ripple current as they charge and discharge. The datasheet specifies a **maximum ripple current in RMS** — this is the current that causes internal heating. Exceeding it shortens the capacitor's life or destroys it. The rating assumes a specific frequency (often 100/120 Hz); at higher frequencies, the allowable ripple current is usually higher because the ESR is lower.

### Transformer ratings (VA)

Transformers are rated in **VA** (volt-amperes), which is $V_{rms} \times I_{rms}$. A 100VA transformer can deliver 100V at 1A, or 50V at 2A, etc. — but only if the waveforms are sinusoidal. Non-sinusoidal currents (from rectifiers, for example) cause additional heating due to harmonics, effectively derating the transformer.

### Audio power: the marketing battlefield

Audio amplifier power can be specified several ways:

| Specification | What it means | Trustworthiness |
|--------------|---------------|----------------|
| **RMS power** | $V_{rms}^2 / R$ sustained into rated load. The real number. | Honest |
| **Peak power** | $V_{peak}^2 / R$ — double the RMS power for sine waves. | Technically valid but misleading |
| **PMPO** (Peak Music Power Output) | A marketing fiction — often 5–10× the RMS rating, calculated under unrealistic conditions. | Ignore |

A "1000W PMPO" speaker system might deliver 50–100W RMS. Always look for the RMS number.

### Noise specifications

Noise on datasheets is almost always specified in **µV_rms** or **nV/√Hz** (noise spectral density). Since noise is random and has no meaningful peak value, RMS is the only sensible way to quantify it. The RMS noise tells you the equivalent DC voltage that would deliver the same power — which is exactly what you need to calculate signal-to-noise ratio.

---
## Summary

| Concept | Key Point |
|---------|-----------|
| General RMS formula | $V_{rms} = \sqrt{\frac{1}{N}\sum v_i^2}$ — works for any signal |
| DC + AC decomposition | $V_{rms,total}^2 = V_{DC}^2 + V_{rms,AC}^2$ |
| PWM | $V_{rms} = V_{high} \times \sqrt{D}$ — NOT $V_{high} \times D$ |
| Full-wave rectified | Same RMS as unrectified — squaring already ignores sign |
| Clipping | Increases RMS — clipped sine approaches square wave |
| Quadrature addition | $V_{rms,total} = \sqrt{V_1^2 + V_2^2}$ for signals at different frequencies |
| Crest Factor | $V_{peak}/V_{rms}$ — determines headroom requirements |
| Form Factor | $V_{rms}/V_{avg(rect)}$ — determines average-responding meter error |
| Average-responding meter | Measures V_avg, multiplies by 1.111. Correct for sine only. |
| True RMS meter | Actually computes √(mean(v²)). Correct for any waveform. |
| Datasheets | Capacitor ripple (RMS), transformer VA (RMS), audio watts (insist on RMS) |

---
## Checkpoint Questions

**Q1:** A PWM signal swings between 0V and 5V with a 30% duty cycle. What is the RMS voltage?

**Q2:** You have a 2V_rms sine wave signal contaminated with 0.5V_rms of noise at a different frequency. What is the total RMS voltage?

**Q3:** A sine wave is clipped at 70% of its peak. Does the RMS go up or down compared to the unclipped sine? Why?

**Q4:** Your average-responding meter (like the MM300) reads "3.54V AC" on a triangle wave. What is the actual RMS voltage of that signal?

**Q5:** A capacitor is rated for 1A ripple current (RMS). You're passing a 2A peak, 50% duty cycle square wave through it. Is the capacitor within spec?

In [ ]:
# --- Checkpoint Answers (run to reveal) ---

print("=" * 60)
print("CHECKPOINT ANSWERS")
print("=" * 60)

# Q1
V_high, D = 5.0, 0.30
v_rms_q1 = V_high * np.sqrt(D)
print(f"\nQ1: V_rms = V_high × √D = {V_high}V × √{D} = {V_high} × {np.sqrt(D):.3f} = {v_rms_q1:.2f}V")

# Q2
v1, v2 = 2.0, 0.5
v_total_q2 = np.sqrt(v1**2 + v2**2)
print(f"\nQ2: V_rms(total) = √(V1² + V2²) = √({v1}² + {v2}²) = √{v1**2 + v2**2:.2f} = {v_total_q2:.3f}V")
print(f"    NOT {v1} + {v2} = {v1+v2}V. RMS values add in quadrature.")

# Q3
print(f"\nQ3: RMS goes UP. Clipping flattens the peaks, making the waveform")
print(f"    more square-like. Since a square wave has a higher RMS-to-peak")
print(f"    ratio than a sine wave (1.0 vs 0.707), the RMS increases.")
print(f"    The clipped portions spend MORE time at high voltage, not less.")

# Q4
mm300_reading = 3.54
SINE_FF = np.pi / (2 * np.sqrt(2))
TRI_FF = 2 / np.sqrt(3)
# MM300 did: V_avg_rect * 1.111 = 3.54, so V_avg_rect = 3.54 / 1.111
v_avg_rect = mm300_reading / SINE_FF
# True RMS = V_avg_rect * triangle_form_factor
v_rms_true = v_avg_rect * TRI_FF
# Or equivalently: V_rms_true = mm300_reading * (TRI_FF / SINE_FF)
correction = TRI_FF / SINE_FF
print(f"\nQ4: The MM300 measured V_avg(rect) = {mm300_reading}V / {SINE_FF:.4f} = {v_avg_rect:.3f}V")
print(f"    The true RMS = V_avg(rect) × triangle_form_factor")
print(f"                 = {v_avg_rect:.3f}V × {TRI_FF:.4f} = {v_rms_true:.3f}V")
print(f"    Or: multiply the MM300 reading by {correction:.4f} = {mm300_reading * correction:.3f}V")

# Q5
I_peak = 2.0
D_q5 = 0.50
I_rms = I_peak * np.sqrt(D_q5)
print(f"\nQ5: I_rms = I_peak × √D = {I_peak}A × √{D_q5} = {I_peak} × {np.sqrt(D_q5):.3f} = {I_rms:.3f}A")
print(f"    The capacitor is rated for 1A RMS ripple current.")
print(f"    {I_rms:.3f}A > 1A → The capacitor is OVER spec. It will overheat.")
print(f"    You need a capacitor rated for at least {I_rms:.2f}A ripple current.")